In [3]:
import datetime
import time

import pandas as pd
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pgpasslib

from querymsc import WorkflowTools
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
connection_string = "postgresql+psycopg2://nannau@dbmsc.pcic.uvic.ca/msc"
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
start_time = datetime.datetime(1990, 1, 1)
end_time = datetime.datetime(1991, 1, 1)

msc = WorkflowTools(start_time, end_time)

1.0 31.0 744.0


In [6]:
annual_tot_precip = msc.query_annual_precip(session)
annual_tot_rain = msc.query_annual_rain(session)
query_temp = msc.query_design_temp_percentile(session)

In [10]:
query = msc.query_design_temp_percentile(session)

In [11]:
annual_tot_precip = msc.query_annual_precip(session)
pd.read_sql(annual_tot_precip.statement, engine).head()

,annual_precip,time_min,time_max,lat,lon,station_id,obs_count,completeness
0,1052.0,1990-01-01,1990-12-31,41.7833,-82.6833,6063,365,1.0
1,1159.7,1990-01-01,1990-12-31,41.9500,-82.5167,6140,365,1.0
2,1061.4,1990-01-01,1990-12-31,42.0333,-82.9000,6036,365,1.0
3,1144.4,1990-01-01,1990-12-31,42.0431,-82.6739,6040,365,1.0
4,1189.5,1990-01-01,1990-12-31,42.1034,-83.0945,5992,365,1.0


In [12]:
annual_tot_rain = msc.query_annual_rain(session)
pd.read_sql(annual_tot_rain.statement, engine).head()

In [ ]:
query_temp = msc.query_design_temp_percentile(session)
pd.read_sql(query_temp.statement, engine).head()

In [ ]:
query_design_temp_dry = msc.query_design_temp_dry(session, month=7)
pd.read_sql(query_design_temp_dry.statement, engine).head()

In [ ]:
query_design_temp_wet = msc.query_design_temp_wet(session, month=7)
pd.read_sql(query_design_temp_wet.statement, engine).head()

In [11]:
df_vars = pd.read_sql(msc.query_variables(session).statement, engine)
#df_vars[df_vars['net_var_name']=='127']
#df_vars[(df_vars['net_var_name'].astype(int) > 124) & (df_vars['net_var_name'].astype(int) < 132)]
#df_vars[df_vars['standard_name'].str.contains('temperature')]